In [8]:
%load_ext autoreload
%autoreload 2

In [38]:
import os 
# os.environ["ROOT"] = "E:\\"
from datetime import datetime, timedelta
try:
    from worker.song_history import load_df
    from worker.song_history import  get_durations
except:
    from song_history import load_df
    from song_history import  get_durations
import pandas as pd 
import plotly.offline as opy
import io, json, logging
import requests as r
import os 
import pickle

pd.options.plotting.backend = "plotly"

In [27]:
from worker.plot import artist_duration
from worker.authorize import get_token

In [3]:
df, mdf = load_df()

loaded from E:/pmhalvor//data/history.csv


In [4]:
df = df[df["played_at"] > (datetime.today() - timedelta(180)).strftime("%Y-%m-%dZ%H:%M:%S.000Z")]


In [5]:
df.head()

,played_at,id,artist,name
28338,2022-02-27T10:40:31.735Z,1No3OKWvdgtctLvSKQcREg,Khruangbin,Chocolate Hills
28339,2022-02-27T10:44:19.058Z,1zEkOtRGRBy85BU2dW0QsZ,Gavinco,Caravella - Edit
28340,2022-02-27T10:46:37.839Z,3EFJiOTKJ14jAWzuYxA46c,Remulak,Drifting
28341,2022-02-27T10:50:01.062Z,2FCQs4zlAjGeIh0KIqkEmD,The Olympians,Sagittarius By Moonlight
28342,2022-02-27T10:57:11.913Z,1hYLj6JNbs8RAwNNy3x3HG,Mark Kramer Trio,Mozart / Jazz Symphony in G Minor First Moveme...


In [6]:
len(df.id.unique())

3670

In [85]:
durations = get_durations(df.id.unique())

3670 new ids to check
Time difference: 2229
Will be executing 74 API call(s)


TypeError: sequence item 0: expected str instance, int found

In [24]:
durations.head()

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0


In [21]:
ids = durations.id[durations.duration<1]

In [22]:
ids

0       1No3OKWvdgtctLvSKQcREg
1       1zEkOtRGRBy85BU2dW0QsZ
2       3EFJiOTKJ14jAWzuYxA46c
3       2FCQs4zlAjGeIh0KIqkEmD
4       1hYLj6JNbs8RAwNNy3x3HG
                 ...          
3665    4bO6DljpuAeQh6HS20i0I5
3666    7cPojUq1xFAIcksvhFp4DV
3667    5Axapk6xMmSHK2A75EZiYo
3668    07ABETRdek3ACMpRPvQuaT
3669    4aeJDu3ftusgtKHpTxjcjk
Name: id, Length: 3670, dtype: object

In [34]:
MAX_ID_COUNT = 10
token = get_token()
batches = (len(ids)//MAX_ID_COUNT) + 1
print(f'Will be executing {batches} API call(s)')

Time difference: 715
Will be executing 368 API call(s)


In [35]:
URL = "https://api.spotify.com/v1/tracks"    # api-endpoint for recently played  
HEAD = {'Authorization': 'Bearer '+token}                       # provide auth. crendtials

In [36]:
durations.head(30)

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0
5,3yMYs3jJdAPxMgqb0CYZUG,0
6,6Rd2NhQnVeDBMUti7BiS90,0
7,5STMWcvgi6UiswwZyeWdTG,0
8,0upn2c2d9jGu6nI6hiTjW9,0
9,5YQxSoa9DJvbRz49X9BTc8,0


In [99]:
dir_path = os.environ["ROOT"]


for i in range(1):
    if i==(batches-1):
        batch_ids = ids[MAX_ID_COUNT*i:]  # last set of indices
    else:
        batch_ids = ids[MAX_ID_COUNT*i:MAX_ID_COUNT*(i+1)]  # forward indexing

    print(f"requesting {len(batch_ids)} ids")
    b_ids = ','.join(batch_ids)
    
    print(b_ids)
    PARAMS = {'ids':b_ids}	
    data = r.get(url=URL, headers=HEAD, params=PARAMS).json()

    batch_dur = []

    if data.get('tracks'):
        for track in data['tracks']:
            batch_dur.append(track['duration_ms'])

#             print(batch_dur)
#         durations.duration[batch_ids.index] = batch_dur
#     else:
#         print('No tracks in response')
#         print(data)

#     print(durations[durations.index.isin(batch_ids)])

requesting 10 ids
1No3OKWvdgtctLvSKQcREg,1zEkOtRGRBy85BU2dW0QsZ,3EFJiOTKJ14jAWzuYxA46c,2FCQs4zlAjGeIh0KIqkEmD,1hYLj6JNbs8RAwNNy3x3HG,3yMYs3jJdAPxMgqb0CYZUG,6Rd2NhQnVeDBMUti7BiS90,5STMWcvgi6UiswwZyeWdTG,0upn2c2d9jGu6nI6hiTjW9,5YQxSoa9DJvbRz49X9BTc8


In [89]:
batch_ids.index

Int64Index([10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype='int64')

In [53]:
durations.duration[batch_ids.index]

20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
Name: duration, dtype: int64

In [100]:
batch_dur

[190433,
 234278,
 145774,
 210240,
 437880,
 380000,
 331392,
 197968,
 379229,
 146341]

In [101]:
dur = pd.DataFrame({"dur": batch_dur})

In [102]:
dur.index = batch_ids.index

In [103]:
dur

,dur
0,190433
1,234278
2,145774
3,210240
4,437880
5,380000
6,331392
7,197968
8,379229
9,146341


In [107]:
durations.duration[batch_ids.index] = batch_dur

C:\Users\perha\AppData\Local\Temp\ipykernel_22972\662607360.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
durations

,id,duration,dur
0,1No3OKWvdgtctLvSKQcREg,190433,NaN
1,1zEkOtRGRBy85BU2dW0QsZ,234278,NaN
2,3EFJiOTKJ14jAWzuYxA46c,145774,NaN
3,2FCQs4zlAjGeIh0KIqkEmD,210240,NaN
4,1hYLj6JNbs8RAwNNy3x3HG,437880,NaN
5,3yMYs3jJdAPxMgqb0CYZUG,380000,NaN
6,6Rd2NhQnVeDBMUti7BiS90,331392,NaN
7,5STMWcvgi6UiswwZyeWdTG,197968,NaN
8,0upn2c2d9jGu6nI6hiTjW9,379229,NaN
9,5YQxSoa9DJvbRz49X9BTc8,146341,NaN


In [59]:
durations.merge(batch_ids = batch_dur

C:\Users\perha\AppData\Local\Temp\ipykernel_22972\912534908.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
durations.head(30)

,id,duration
0,1No3OKWvdgtctLvSKQcREg,0
1,1zEkOtRGRBy85BU2dW0QsZ,0
2,3EFJiOTKJ14jAWzuYxA46c,0
3,2FCQs4zlAjGeIh0KIqkEmD,0
4,1hYLj6JNbs8RAwNNy3x3HG,0
5,3yMYs3jJdAPxMgqb0CYZUG,0
6,6Rd2NhQnVeDBMUti7BiS90,0
7,5STMWcvgi6UiswwZyeWdTG,0
8,0upn2c2d9jGu6nI6hiTjW9,0
9,5YQxSoa9DJvbRz49X9BTc8,0
